Practical Part,
Question 2;

Cüneyt Erem,
Ulvi Shukurzade,
Ewald Bindereif,
Bilge Ulusay

In [2]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torchvision.transforms as transforms
import numpy as np

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data_cifar', train=True, download=True, transform = transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data_cifar', train=False, download=True, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 256, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 256, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class net(nn.Module):
    def __init__(self, input_num, hidden, output):
        super(net, self).__init__()
        self.input_num = input_num
        self.dense1 = nn.Linear(input_num, hidden[0])
        self.dense2 = nn.Linear(hidden[0], hidden[1])
        self.dense3 = nn.Linear(hidden[1], output)     
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(x.size(0), self.input_num)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        return x

In [5]:
DEVICE = torch.device('cuda')
model = net(32*32*3, (100, 30), 10).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [6]:
for i in range(25):
    loss_rate = 0.
    for image_train, label_train in train_loader:
        image_train = image_train.float().to(DEVICE)
        label_train = label_train.to(DEVICE)
        optimizer.zero_grad()
        output_label = model(image_train)
        loss = criterion(output_label, label_train)
        loss.backward()
        optimizer.step()
        loss_rate += loss.item()*image_train.size(0)
    loss_rate = loss_rate/len(train_loader.dataset)
    print("epoch: {} and loss: {}".format(i+1, loss_rate))

epoch: 1 and loss: 2.2759455530548096
epoch: 2 and loss: 2.197988823928833
epoch: 3 and loss: 2.101746180648804
epoch: 4 and loss: 2.0205129978942873
epoch: 5 and loss: 1.9629383893585206
epoch: 6 and loss: 1.923136226234436
epoch: 7 and loss: 1.8945437591171264
epoch: 8 and loss: 1.8718334380340576
epoch: 9 and loss: 1.8515281462860107
epoch: 10 and loss: 1.8314786031341552
epoch: 11 and loss: 1.8147729556274415
epoch: 12 and loss: 1.7983118844223023
epoch: 13 and loss: 1.7851604546356201
epoch: 14 and loss: 1.7710650454711914
epoch: 15 and loss: 1.7591964093780517
epoch: 16 and loss: 1.7471750217056274
epoch: 17 and loss: 1.7353364950942993
epoch: 18 and loss: 1.7257641005706787
epoch: 19 and loss: 1.7145941128540039
epoch: 20 and loss: 1.7031203366851806
epoch: 21 and loss: 1.692553325881958
epoch: 22 and loss: 1.6812345377349853
epoch: 23 and loss: 1.6730034964370728
epoch: 24 and loss: 1.663301162147522
epoch: 25 and loss: 1.6537054229736328


In [15]:
model.eval()
true_num = [0.]*10
total_num = [0.]*10
loss_rate2 = 0.

for image_test, label_test in test_loader:
    image_test = image_test.float().to(DEVICE)
    label_test = label_test.to(DEVICE)
    optimizer.zero_grad()
    output_label = model(image_test)
    loss_rate2 += loss.item()*image_test.size(0)
    _, label_pred = torch.max(output_label, 1)
    label_pred = label_pred.cpu().numpy()
    label_correct_rate = torch.tensor([True if label_pred[i] == label_test[i] else False for i in range(len(label_test))]).cpu().numpy()
    for i in range(len(label_test)):
        true_num[label_test.data[i]] += label_correct_rate[i].item()
        total_num[label_test.data[i]] += 1
for i in range(10):
    if total_num[i] > 0:
        print("number: {} and accuracy: {}".format(i, true_num[i]/total_num[i]))
print("total accuracy score: {}".format(accuracy_score(label_correct_rate, label_pred)))
loss_rate2 = loss_rate2/len(test_loader.dataset)
print("total loss: {}".format(loss_rate2))

number: 0 and accuracy: 0.511
number: 1 and accuracy: 0.418
number: 2 and accuracy: 0.309
number: 3 and accuracy: 0.072
number: 4 and accuracy: 0.361
number: 5 and accuracy: 0.271
number: 6 and accuracy: 0.504
number: 7 and accuracy: 0.583
number: 8 and accuracy: 0.471
number: 9 and accuracy: 0.521
total accuracy score: 0.0625
total loss: 1.6280784606933594


Explanation;

Here, we set our model as input with 32*32 with 3 neuron to regulate it for input size, 2 hidden layers with 100 and 30 to keep in same number of layers but increase the number of neurones to show effects of number of neurons and same size 10 neurons for output. Then we used the same learning rate 0.01 and SDG optimization, Relu activation function to optimize in better between 0 and 1 interval than sigmoid. If we incerase the batch size to 256 or number of hidden layers by 1, then performance inreases a bit. Still our accuracy is a bit better than signle linear classification because MLP increases the learning rate, however selecting the correct number of inputs, hidden layers, batch sizes are important so that it is not best result by these results. 